In [53]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import Input, Reshape
from keras.layers.convolutional import UpSampling1D, Conv1D

### get_generative

#### Usando Models

In [85]:
def get_generative(G_in, dense_dim=200, out_dim=50, lr=1e-3):
    x = Dense(dense_dim)(G_in)
    x = Activation('tanh')(x)
    G_out = Dense(out_dim, activation='tanh')(x)
    G = Model(G_in, G_out)
    opt = SGD(lr=lr)
    G.compile(loss='binary_crossentropy', optimizer=opt)
    return G, G_out

G_in = Input(shape=[10])
G, G_out = get_generative(G_in)
G.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 10)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 200)               2200      
_________________________________________________________________
activation_12 (Activation)   (None, 200)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 50)                10050     
Total params: 12,250
Trainable params: 12,250
Non-trainable params: 0
_________________________________________________________________


#### Usando Sequential

In [86]:
model = Sequential()
model.add(Dense(200, input_shape=(10,),activation='linear'))
model.add(Activation('tanh'))
model.add(Dense(50,activation='linear'))
model.compile(optimizer=SGD(lr=1e-3),loss='binary_crossentropy')

In [87]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 200)               2200      
_________________________________________________________________
activation_13 (Activation)   (None, 200)               0         
_________________________________________________________________
dense_53 (Dense)             (None, 50)                10050     
Total params: 12,250
Trainable params: 12,250
Non-trainable params: 0
_________________________________________________________________


In [88]:
model.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52_input (InputLayer)  (None, 10)                0         
_________________________________________________________________
dense_52 (Dense)             (None, 200)               2200      
_________________________________________________________________
activation_13 (Activation)   (None, 200)               0         
_________________________________________________________________
dense_53 (Dense)             (None, 50)                10050     
Total params: 12,250
Trainable params: 12,250
Non-trainable params: 0
_________________________________________________________________


### get_discriminative

#### Usando models

In [89]:
def get_discriminative(D_in, lr=1e-3, drate=.25, n_channels=50, conv_sz=5, leak=.2):
    x = Reshape((-1, 1))(D_in)
    x = Conv1D(n_channels, conv_sz, activation='relu')(x)
    x = Dropout(drate)(x)
    x = Flatten()(x)
    x = Dense(n_channels)(x)
    D_out = Dense(2, activation='sigmoid')(x)
    D = Model(D_in, D_out)
    dopt = Adam(lr=lr)
    D.compile(loss='binary_crossentropy', optimizer=dopt)
    return D, D_out

D_in = Input(shape=[50])
D, D_out = get_discriminative(D_in)
D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 50)                0         
_________________________________________________________________
reshape_15 (Reshape)         (None, 50, 1)             0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 46, 50)            300       
_________________________________________________________________
dropout_12 (Dropout)         (None, 46, 50)            0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 2300)              0         
_________________________________________________________________
dense_54 (Dense)             (None, 50)                115050    
_________________________________________________________________
dense_55 (Dense)             (None, 2)                 102       
Total para

#### Usando Sequential

In [90]:
model = Sequential()
model.add(Reshape((-1, 1), input_shape=(50,)))
model.add(Conv1D(50, 5))
model.add(Dropout(.25))
model.add(Flatten())
model.add(Dense(50))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=Adam(lr=1e-3),loss='binary_crossentropy')

In [91]:
model.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_16_input (InputLayer (None, 50)                0         
_________________________________________________________________
reshape_16 (Reshape)         (None, 50, 1)             0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 46, 50)            300       
_________________________________________________________________
dropout_13 (Dropout)         (None, 46, 50)            0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 2300)              0         
_________________________________________________________________
dense_56 (Dense)             (None, 50)                115050    
_________________________________________________________________
dense_57 (Dense)             (None, 2)                 102       
Total para

### Teste

In [92]:
def get_gerador():
    model = Sequential()
    model.add(Dense(200, input_shape=(10,),activation='linear'))
    model.add(Activation('tanh'))
    model.add(Dense(50,activation='linear'))
    model.compile(optimizer=SGD(lr=1e-3),loss='binary_crossentropy')
    return model

In [94]:
G = get_gerador()
G.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 200)               2200      
_________________________________________________________________
activation_14 (Activation)   (None, 200)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 50)                10050     
Total params: 12,250
Trainable params: 12,250
Non-trainable params: 0
_________________________________________________________________


In [95]:
def get_discriminador():
    model = Sequential()
    model.add(Reshape((-1, 1), input_shape=(50,)))
    model.add(Conv1D(50, 5))
    model.add(Dropout(.25))
    model.add(Flatten())
    model.add(Dense(50))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(optimizer=Adam(lr=1e-3),loss='binary_crossentropy')
    return model    

In [96]:
D = get_discriminador()
D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_17 (Reshape)         (None, 50, 1)             0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 46, 50)            300       
_________________________________________________________________
dropout_14 (Dropout)         (None, 46, 50)            0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 2300)              0         
_________________________________________________________________
dense_60 (Dense)             (None, 50)                115050    
_________________________________________________________________
dense_61 (Dense)             (None, 2)                 102       
Total params: 115,452
Trainable params: 115,452
Non-trainable params: 0
_________________________________________________________________


In [98]:
def set_trainability(model, trainable=False):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable

In [106]:
set_trainability(D, False)